In [4]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random
import operator
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

path_titles_meta = "acl-metadata.txt"




f = open(path_titles_meta, "r", encoding="Latin-1")

# compile titles into a list
titles = []
titlesDict = {}
# list for tokenized documents in loop
texts = []

#Extracting the titles from our meta-document
for line in f:
    if line.startswith('title'):
        tmp = line.split(' = ',2)
        title = tmp[1][1:len(tmp[1])-2] #last part: subtracts /n}; 
        titles.append(title)
        titlesDict[title] = None



# loop through title list    
for i in titles:
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    #show tokenized words
    #print(tokens)
    
    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
   
    
    # stem token "-e are lost??" 
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    #print(stemmed_tokens)
    
    # add tokens to list
    texts.append(stopped_tokens)
    
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)



# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]
print(max([v for s in corpus for v, _ in s]))

# generate LDA model
#ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary, passes=20)

#print(ldamodel.print_topics(num_topics=5, num_words=5))


#print(len(titlesDict.keys()))







    

    

10434


In [5]:
from scipy.sparse import csr_matrix
dictionary_len = max([v for s in corpus for v, _ in s]) + 1
document_len = len(corpus)
converted_input = csr_matrix((document_len, dictionary_len), dtype=np.int16).toarray()
print(converted_input.shape)
tup_dict = corpus

for i in range(len(tup_dict)):
    for (word_idx, count) in tup_dict[i]:
        converted_input [i, word_idx] = count
        
corpus = converted_input

(19424, 10435)


In [6]:
vocab = []
for i in range(len(dictionary.keys())):
    vocab.append(dictionary.get(i))

In [7]:
import lda
import lda.datasets


model = lda.LDA(n_topics=10, n_iter=500, random_state=1)
model.fit(corpus)
topic_word = model.topic_word_  # model.components_ also works
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-n_top_words:-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))
    

Topic 0: translation machine statistical based system phrase state
Topic 1: extraction using based information entity learning text
Topic 2: resolution task system coreference annotation evaluation english
Topic 3: semantic discourse role using learning expressions relations
Topic 4: evaluation automatic text annotation using detection error
Topic 5: speech word based segmentation chinese recognition tagging
Topic 6: parsing dependency grammars grammar based tree using
Topic 7: dialogue system language spoken systems speech question
Topic 8: language natural processing computational grammar generation semantics
Topic 9: word sense using semantic corpora disambiguation lexical


In [8]:
cls = model.transform(corpus)

In [9]:
print(str(titles[0]) + " " + str(sum(cls[0])))

On Dual Decomposition and Linear Programming Relaxations for Natural Language Processing 1.0


In [10]:
print(cls)

[[  9.90432999e-04   6.39876055e-04   7.07266215e-04 ...,   4.92531463e-03
    3.96640773e-01   3.36868867e-05]
 [  3.05528806e-03   4.51801220e-03   2.15391510e-05 ...,   2.04331320e-03
    3.81994719e-04   6.97641261e-03]
 [  8.42137999e-03   3.08472390e-02   1.53655111e-03 ...,   3.59701412e-01
    9.08223080e-04   5.96388551e-02]
 ..., 
 [  5.07129135e-01   7.61880853e-04   1.25600907e-02 ...,   3.71346724e-02
    3.29966240e-02   3.86706593e-03]
 [  5.30792150e-01   1.46837784e-04   1.44271382e-02 ...,   1.19263660e-02
    3.89206178e-02   7.79009494e-02]
 [  6.08861657e-01   2.43025257e-03   2.92429517e-04 ...,   6.44618565e-04
    1.31408335e-04   7.21876738e-02]]


In [11]:
path_titles_meta_lda = "acl-metadata-lda.txt"


f = open(path_titles_meta_lda, "w", encoding="Latin-1")

for i in range(len(titles)):
    f.write(str(titles[i]) + "\t" + str(cls[i]) + "\n")
f.close()

